In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time

In [24]:
NB_Hotel_MAX = 50
EXPORT_PATH = "./exports/"

In [25]:
url = "https://www.pretapartir.fr/recherche#!/search?th=SJ&sort=promo_desc&depart=A_PAR&f=3_AL&dureeNuit=7-99"

options = webdriver.ChromeOptions()
#options.add_argument('headless')
browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver",options=options)

browser.get(url)

time.sleep(2) #wait till all the content will be charged

##-------------##

#get the nb of results 

soup = BeautifulSoup(browser.page_source, 'html.parser') #first param: page html; 2 param: type of parser

nb_elements = int(soup.find("h1").find("strong").get_text(strip = True)) #strip = true: remove all other characters
#nb_elements

##------------------##

#define the nb of click to load new pages

nb_click = NB_Hotel_MAX /10 -1 if nb_elements >= NB_Hotel_MAX else nb_elements/10
#nb_click

#----------#

for i in range(0,int(nb_click)):
    button_next_10 = browser.find_element(By.CSS_SELECTOR,"a[ng-click='vm.loadMorePackage()']")
    #button_next_10.click()
    
    browser.execute_script("arguments[0].click()",button_next_10)
    time.sleep(2)

soup = BeautifulSoup(browser.page_source, 'html.parser') 

browser.quit() #quit the navigator


#----------#



/tmp/ipykernel_23312/98850383.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver",options=options)


In [26]:
# Get items
items = soup.find_all("div",{"class":"blocProduct-wrapper ng-scope"})
#print(len(items))


50


In [33]:
def clean_text(str):
    return str.replace("\n","").replace("\t","")

In [43]:
import re
item = items[0]
country = clean_text(item.find("p",{"class":"blocProduct-place"}).get_text(strip = True)).replace(" ","")
Hotel = clean_text(item.find("p",{"class":"blocProduct-title"}).get_text(strip = True))

details = item.find("div",{"class":"blocProduct-detail"}).findChildren()

duration = details[0].get_text() 
date = re.search('([0-9]{2}\/[0-9]{2}\/[0-9]{4})',details[1].get_text()).group(1)

price = item.find("div", {"class": "blocProduct-price"}).findChildren()[1].get_text()
price

'539€ / pers'

In [ ]:
# store items scraped into a dict inside a list
travels = []
for item in items:
    country = clean_text(item.find("p",{"class":"blocProduct-place"}).get_text(strip = True)).replace(" ","")
    hotel = clean_text(item.find("p",{"class":"blocProduct-title"}).get_text(strip = True))

    details = item.find("div",{"class":"blocProduct-detail"}).findChildren()

    duration = details[0].get_text() 
    date = re.search('([0-9]{2}\/[0-9]{2}\/[0-9]{4})',details[1].get_text()).group(1)

    price = item.find("div", {"class": "blocProduct-price"}).findChildren()[1].get_text()

    travel = {
        "country":country,
        "hotel":hotel,
        "duration":duration,
        "date":date,
        "price":price,

    }
    travels.append(travel)


#travels

In [ ]:
#export data into excel file
import pandas as pd

df = pd.DataFrame(travels)
df


In [53]:
df.to_excel(EXPORT_PATH + 'holidays.xlsx', index = False)